In [1]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/project2/Lie_algebras-interpretability-/"

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/project2/Lie_algebras-interpretability-


In [2]:
pip install wandb

In [ ]:
from train_sae import main

model=main()

Using device: cpu
Number of trainable parameters: 165259776
autoencoders.0.W_encode: torch.Size([768, 53760])
autoencoders.0.bias_encode: torch.Size([53760])
autoencoders.0.W_decode: torch.Size([53760, 768])
autoencoders.0.bias_decode: torch.Size([768])
autoencoders.1.W_encode: torch.Size([768, 53760])
autoencoders.1.bias_encode: torch.Size([53760])
autoencoders.1.W_decode: torch.Size([53760, 768])
autoencoders.1.bias_decode: torch.Size([768])
Losses: (tensor(2291.5981, grad_fn=<DivBackward0>), (tensor(1.8782, grad_fn=<DivBackward0>), tensor(2290.4927, grad_fn=<DivBackward0>), tensor(0.6667, grad_fn=<DivBackward0>)))


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
from merge import MergedModel
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments
from sae_direction_alignment import AutoencoderMerged
from evaluate import MergedModelArguments

def get_token_autoencoder_representation(token, merged_model, dataset, layer, device):
    #  token: int, token to investigate
    #  merged_model: MergedModel, model to use for representation
    #  dataset: Dataloader, dataset to avergae the representation over
    #  returns: (torch.Tensor(num_of_models), latent_dim), (average representation of the token for each model merging,
    #  average representation of the token for the merged model (essentialy the average over the first output))
    res=(None, None)
    num_of_appearances_so_far=0

    for batch in dataset:
        with torch.no_grad():
            input_ids = batch.to(device)
            if (input_ids == token).any():
                indices = (input_ids == token).nonzero(as_tuple=True)
                print(input_ids)
                indices_tuples = list(zip(*indices))
                print(indices_tuples)
                output = merged_model(batch, return_embeddings=True)[1][layer]
                if res[0] is None:
                    res = [torch.zeros(len(output[0]), output[0][0].shape[-1]),
                           torch.zeros(output[0][0].shape[-1])]
                print()
                print(output[0].shape, output[1].shape)
                output = (torch.cat([sum([output[0][j][index] for index in indices_tuples]).unsqueeze(0) for j in range(len(merged_model.models))], dim=0),
                          sum([output[1][index] for index in indices_tuples]))
                print(output[0])
                res[0]=res[0]*num_of_appearances_so_far + output[0]/(num_of_appearances_so_far+len(indices_tuples))
                res[1]=res[1]*num_of_appearances_so_far + output[1]/(num_of_appearances_so_far+len(indices_tuples))
                num_of_appearances_so_far+=len(indices_tuples)

    return res

class TextDataset(Dataset):
  def __init__(self, file_path, tokenizer, max_len):
    super(TextDataset, self).__init__()
    self.tokenizer = tokenizer
    self.max_len = max_len
    with open(file_path, 'r', encoding = 'utf-8') as file:
      self.lines = file.readlines()
    self.lines = [line.strip() for line in self.lines if line.strip()]

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx) :
    line = self.lines[idx].strip()
    encoding = self.tokenizer(line, truncation = True, max_length = self.max_len, padding = 'max_length', return_tensors = 'pt')
    return encoding['input_ids'].squeeze()


def load_dataset(file_path, tokenizer, max_len, batch_size):
    dataset = TextDataset(file_path, tokenizer, max_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

def load_merged_model(model_path, cfg, models, layer, device):
    merged_model = AutoencoderMerged(models, cfg, layer=layer, device=device)
    merged_model.load_state_dict(torch.load(model_path, map_location=device))
    merged_model.eval()
    return merged_model

model_name1 = "Sharathhebbar24/math_gpt2_sft"
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
model1 = AutoModelForCausalLM.from_pretrained(model_name1).to('cpu')

model_name2 = "yoavgur/gpt2-bash-history-baseline"
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForCausalLM.from_pretrained(model_name2).to('cpu')

class CustomEvaluateArguments(TrainingArguments):
    def __init__(self, *args, multiplier=1, d_models=[], lambda_l1=1, lambda_cos=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.multiplier = multiplier
        self.d_models = d_models
        self.lambda_l1 = lambda_l1
        self.lambda_cos = lambda_cos

cfg = CustomEvaluateArguments(
    output_dir='./results',
    multiplier=70,
    d_models=[768, 768],
    lambda_l1=0.5,
    lambda_cos=0.5,
)



res=get_token_autoencoder_representation(4, model, eval_dataloader, layer=-2, device='cpu')

tensor([[ 1212,   318,  3478,  7319,  8714,  1123,   284,   530,  3470,  1510,
          7183,    11, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256],
        [ 4758,   318,   838,     4,   286,   262,  2938,  1271,   286,  3644,
          2985,   416,   262,   886, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256]])
[(tensor(1), tensor(3))]

torch.Size([2, 2, 32, 53760]) torch.Size([2, 32, 53760])
tensor([[0.0000, 0.0000, 0.4044,  ..., 0.3695, 0.0000, 0.0000],
        [0.0000, 0.2914, 0.0000,  ..., 0.1374, 0.0738, 0.7009]])


In [15]:
res

[tensor([[0.0000, 0.0000, 0.4044,  ..., 0.3695, 0.0000, 0.0000],
         [0.0000, 0.2914, 0.0000,  ..., 0.1374, 0.0738, 0.7009]]),
 tensor([0.0000, 0.1457, 0.2022,  ..., 0.2535, 0.0369, 0.3505])]

In [16]:
from evaluate import main as evaluate_main

evaluate_main()

cpu


TypeError: super(type, obj): obj must be an instance or subtype of type

In [4]:
from merge import MergedModel, MergedModelArguments
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import TrainingArguments
from sae_direction_alignment import AutoencoderMerged

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CustomEvaluateArguments(TrainingArguments):
    def __init__(self, *args, multiplier=1, d_models=[], lambda_l1=1, lambda_cos=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.multiplier = multiplier
        self.d_models = d_models
        self.lambda_l1 = lambda_l1
        self.lambda_cos = lambda_cos

def load_merged_model(model_path, cfg, models, layer, device):
    merged_model = AutoencoderMerged(models, cfg, layer=layer, device=device)
    merged_model.load_state_dict(torch.load(model_path, map_location=device))
    merged_model.eval()
    return merged_model

cfg = CustomEvaluateArguments(
    output_dir='./results',
    multiplier=70,
    d_models=[768, 768],
    lambda_l1=0.5,
    lambda_cos=0.5,
)

model_name1 = "Sharathhebbar24/math_gpt2_sft"
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
model1 = AutoModelForCausalLM.from_pretrained(model_name1).to(device)

model_name2 = "yoavgur/gpt2-bash-history-baseline"
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForCausalLM.from_pretrained(model_name2).to(device)

merged_model = load_merged_model('./results/model_epoch_-1_layer5_loss_88489494.0000.pt', cfg, [model1, model2], layer=-2, device=device)

mergedcfg=MergedModelArguments()

model=MergedModel([model1, model2], mergedcfg, device, [merged_model])

test_input = tokenizer1(["That beauty still may live in thine or thee. I", "This is another test"], return_tensors='pt', padding=True)['input_ids'].to(device)

ans=model.generate(test_input, max_tokens=30)

print(tokenizer1.decode(ans[0]))
print(tokenizer1.decode(ans[1]))

 have a daughter, and she is a beautiful and well-liked girl.
             
|

The answer is: 

                    
